In [2]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

#### Q1. Define function description

In [3]:
get_weather_tool = {
  "type": "function",
  "function": {
    "name": "get_weather",
    "description": "Get the current weather for a given city",
    "parameters": {
      "type": "object",
      "properties": {
        "city": {
          "type": "string",
          "description": "Name of the city to get the weather for"
        }
      },
      "required": ["city"],
      "additionalProperties": False
    }
  }
}

In [4]:
import chat_assistant
from chat_assistant import Tools, ChatAssistant, ChatInterface
import os

tools = Tools()
tools.add_tool(get_weather, get_weather_tool)

interface = ChatInterface()

developer_prompt = """
You're a helpful weather bot. Use the get_weather tool to answer questions about weather.
"""

# Use your Groq client here
from groq import Groq
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

chat = ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=interface,
    client=client
)


In [4]:
chat.run()

BadRequestError: Error code: 400 - {'error': {'message': "'messages.3' : input does not contain the discriminator property 'role'", 'type': 'invalid_request_error'}}

In [5]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [6]:
set_weather_tool = {
    "type": "function",
    "function": {
        "name": "set_weather",
        "description": "Add or update the temperature for a city in the weather database.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city to set the temperature for."
                },
                "temp": {
                    "type": "number",
                    "description": "The temperature in Celsius to associate with the city."
                }
            },
            "required": ["city", "temp"],
            "additionalProperties": False
        }
    }
}


#### Q3. Install FastMCP


In [7]:
!pip install fastmcp

  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 31.2 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11103 sha256=ebcb1134d69ef87825dd8306f133c2feb32b0592cdb7a84334663754d60d6ecf
  Stored in directory: /home/codespace/.cache/pip/wheels/e0/e8/fc/8ab8aa326e33bc066ccd5f3ca9646eab4299881af933f94f09
Successfully built pyperclip
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.2
    Uninstalling pydantic_core-2.27.2:
      Successful

In [8]:
!pip show fastmcp


Name: fastmcp
Version: 2.10.5
Summary: The fast, Pythonic way to build MCP servers and clients.
Home-page: https://gofastmcp.com
Author: Jeremiah Lowin
Author-email: 
License-Expression: Apache-2.0
Location: /usr/local/python/3.12.1/lib/python3.12/site-packages
Requires: authlib, cyclopts, exceptiongroup, httpx, mcp, openapi-pydantic, pydantic, pyperclip, python-dotenv, rich
Required-by: 


#### Q4. Simple MCP Server


#### Starting MCP server 'Demo 🚀' with transport server.py:1371                              'stdio'  

In [ ]:
# weather_server.py
import random
from fastmcp import FastMCP

# Simulated weather database
known_weather_data = {
    'berlin': 20.0
}

# Initialize MCP server
mcp = FastMCP("Demo 🚀")

@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()
    if city in known_weather_data:
        return known_weather_data[city]
    return round(random.uniform(-5, 35), 1)

@mcp.tool
def set_weather(city: str, temp: float) -> str:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return "OK"

# Run the server
if __name__ == "__main__":
    mcp.run()


#### Q5. Protocol

#### {"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}

#### Q6. Client

In [8]:
from fastmcp import Client
import weather_server  # assuming you already ran weather_server.mcp.run()

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print("Available tools:")
        for tool in tools:
            print(tool)

# Run it in Jupyter
await main()


Available tools:
name='get_weather' title=None description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.' inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True} annotations=None meta=None
name='set_weather' title=None description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update." inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 